In [10]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.metrics.pairwise import cosine_similarity
import math
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import TweetTokenizer
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)

#sql_select = '''select created_at, location, tweet_content, cleaned_tweet, vector from tweets ORDER BY RANDOM() LIMIT 300000;'''
sql_select = '''select created_at, location, tweet_content, cleaned_tweet, hashtags from tweets where hashtags != 'None';'''
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.reset_index(inplace = True)

In [11]:
df['created_datetime'] = pd.to_datetime(df['created_at'])
df['year'] = df.created_datetime.apply(lambda x: x.year)
df['month'] = df.created_datetime.apply(lambda x: x.month)
df['day'] = df.created_datetime.apply(lambda x: x.day)
df['dayofweek'] = df.created_datetime.apply(lambda x: x.dayofweek)
df['hour'] = df.created_datetime.apply(lambda x: x.hour)

In [12]:
df

,index,cleaned_tweet,created_at,hashtags,location,tweet_content,created_datetime,year,month,day,dayofweek,hour
0,0,even stranger is finding any true experts at t...,Wed Jun 07 02:27:44 +0000 2017,drones,"burbank, ca",even stranger is finding any true experts at t...,2017-06-07 02:27:44,2017,6,7,2,2
1,1,Stand By Myanmar,Wed May 31 23:54:39 +0000 2017,makeamovieforeign,"los angeles, ca",stand by myanmar #makeamovieforeign,2017-05-31 23:54:39,2017,5,31,2,23
2,2,summer is here!! it's braid season #boxbraidsb...,Wed Jun 07 02:27:44 +0000 2017,boxbraidsbyme flawless,"los angeles, ca",🌞summer is here!! it's braid season #boxbraids...,2017-06-07 02:27:44,2017,6,7,2,2
3,3,my brothers driving has me soooooo dizzy 🤢,Wed Jun 07 02:27:45 +0000 2017,boxbraidsbyme flawless,"california, usa",my brothers driving has me soooooo dizzy 🤢,2017-06-07 02:27:45,2017,6,7,2,2
4,4,"i'm late... i know. #itfdb also, i was just y...",Wed Jun 07 02:27:46 +0000 2017,itfdb,"los angeles, ca","i'm late... i know. 😫 #itfdb also, i was just ...",2017-06-07 02:27:46,2017,6,7,2,2
5,5,Don't let fool you. William F Buckley Jr was a...,Wed May 31 23:54:25 +0000 2017,hardball,"los angeles, ca",don't let #hardball fool you. william f buckle...,2017-05-31 23:54:25,2017,5,31,2,23
6,6,The encompassed within hip- hop is beyond the,Wed May 31 23:54:29 +0000 2017,energy audible spectrum,"los angeles, ca",the #energy encompassed within hip- hop is bey...,2017-05-31 23:54:29,2017,5,31,2,23
7,7,LA Zoo Today 🙈🐨🐭🦇🐼🦊 . . . .,Wed May 31 23:54:30 +0000 2017,beverlyhills90210 losangeles lifestyle lazoo ...,"los angeles, ca",la zoo today 🙈🐨🐭🦇🐼🦊 . . . . #beverlyhills90210...,2017-05-31 23:54:30,2017,5,31,2,23
8,8,❤ thank you!!!!,Wed Jun 07 02:27:48 +0000 2017,itfdb,"burbank, ca",@matchapon_ ❤ thank you!!!!,2017-06-07 02:27:48,2017,6,7,2,2
9,9,hollow man giggs,Wed Jun 07 02:27:49 +0000 2017,itfdb,"los angeles, ca",hollow man giggs,2017-06-07 02:27:49,2017,6,7,2,2


In [5]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')
hashtag_tfidf = tfidf_vectorizer.fit_transform(df['hashtags'].values.astype('U'))

In [7]:
import pickle
!pip install redis
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)
hashtag_tfidf = pickle.dumps(hashtag_tfidf)
r.set('hashtag_tfidf', hashtag_tfidf)
r.keys()

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


[b'tweet_tfidf_transform',
 b'tweet_tfidf_model',
 b'tweet_tfidf_model_fit',
 b'tweet_SVD_model_fit',
 b'hashtag_tfidf',
 b'SVD_model_fit']

In [ ]:
#r.flushall()